In [1]:
import requests
import mysql.connector
from datetime import datetime

In [3]:
def connect_db():
    return mysql.connector.connect(
            host='localhost',
            database='Restaurant',
            user='root',
            password='root'
        )

In [21]:
def put_data_in_bdd(url):
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Vérifier si la clé "results" existe et contient des données
        if "results" not in data or not data["results"]:
            print("Aucun restaurant trouvé dans la réponse.")
            return False  # Sortir de la fonction si "results" est vide ou inexistant

        # Connexion à la base de données
        connection = connect_db()
        cursor = connection.cursor()

        for record in data["results"]:
            name = record.get("name")

            # Vérification que le nom n'est pas nul ou vide
            if not name:
                print("Nom du restaurant est nul ou vide, saut de cet enregistrement.")
                continue  # Passer à l'itération suivante si le nom est invalide

            food_type = record.get("type")

            # Assurer que cuisine est toujours une liste
            cuisine = record.get("cuisine", [])
            if not isinstance(cuisine, list):
                cuisine = [cuisine] if cuisine else []  # Si cuisine n'est pas une liste, fais-en une liste ou une liste vide

            # Joindre la liste en une chaîne
            cuisine = ", ".join(cuisine) if cuisine else None

            meta_code_com = record.get("meta_code_com")
            meta_code_dep = record.get("meta_code_dep")
            meta_geo_point_lat = record.get("meta_geo_point", {}).get("lat")
            meta_geo_point_lon = record.get("meta_geo_point", {}).get("lon")
            created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Timestamp actuel

            # Vérifier si le restaurant propose des options véganes ou végétariennes
            vegan = 1 if record.get("vegan") == "yes" else 0
            vegetarian = 1 if record.get("vegetarian") == "yes" else 0

            # Vérifier si le restaurant avec le même nom et la même localisation existe déjà
            cursor.execute("""
                SELECT * FROM Restaurants 
                WHERE name = %s AND meta_geo_point_lat = %s AND meta_geo_point_lon = %s
            """, (name, meta_geo_point_lat, meta_geo_point_lon))
            existing_record = cursor.fetchone()

            if existing_record:
                print(f"Restaurant '{name}' à cette localisation existe déjà dans la base de données.")
            else:
                # Insérer de nouvelles données
                insert_query = """
                    INSERT INTO Restaurants
                    (name, type, cuisine, meta_code_com, meta_code_dep, meta_geo_point_lat, meta_geo_point_lon, created_at, vegan, vegetarian)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (name, food_type, cuisine, meta_code_com, meta_code_dep, meta_geo_point_lat, meta_geo_point_lon, created_at, vegan, vegetarian))
                connection.commit()

        # Fermer la connexion à la base de données
        cursor.close()
        connection.close()
        return True

    else:
        print("Échec de la récupération des données")
        return False


In [27]:
def fetch_restaurants():
    base_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/osm-france-food-service/records"

    for arrondissement in range(75101, 75121):
        params = {
            "where": f"meta_code_com = {arrondissement}",
            "limit": 100,
            "refine": "type:\"restaurant\""
        }

        offset = 0
        while True:
            params["offset"] = offset
            url = f"{base_url}?where={params['where']}&limit={params['limit']}&offset={offset}&refine={params['refine']}"
            if not put_data_in_bdd(url):
                break
            print(f"Données insérées pour l'arrondissement {arrondissement}, offset {offset}")

            # Avancer au prochain lot
            offset += 100


In [29]:
fetch_restaurants()

Restaurant 'Kanadé' à cette localisation existe déjà dans la base de données.
Restaurant 'Balagan' à cette localisation existe déjà dans la base de données.
Restaurant 'Kodawari Ramen (Tsukiji)' à cette localisation existe déjà dans la base de données.
Restaurant 'Baan Boran' à cette localisation existe déjà dans la base de données.
Restaurant 'La Poule au Pot' à cette localisation existe déjà dans la base de données.
Restaurant 'Eats Thyme' à cette localisation existe déjà dans la base de données.
Restaurant 'Les Marmottes' à cette localisation existe déjà dans la base de données.
Restaurant 'Bigo' à cette localisation existe déjà dans la base de données.
Restaurant 'Le Royal Opéra' à cette localisation existe déjà dans la base de données.
Restaurant 'Higuma' à cette localisation existe déjà dans la base de données.
Restaurant 'Hakata' à cette localisation existe déjà dans la base de données.
Restaurant 'Café Rive Droite' à cette localisation existe déjà dans la base de données.
Resta